In [1]:
%pip install networkx matplotlib seaborn pulp
from pulp import LpProblem
import pandas as pd    
import matplotlib.pyplot as plt  
import seaborn as sns
from pulp import *
import pickle
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import random
import csv
import plotly.express


Note: you may need to restart the kernel to use updated packages.


In [2]:
#import Data of Arrival flight 
flights = pd.read_excel("Radarfilght_Tuesday_Dec_19.xlsx",)
flights.insert(0, 'Flights_no', range(1, 1 + len(flights)))
flights

,Flights_no,Arrival_time,Flight_name,From,Airline,Aircraft,ICAO code,plane_size,type
0,1,03:00:00,VZ3579,Zhengzhou (CGO),Thai VietJet Air,320,C,3,International
1,2,04:50:00,3U3767,Chengdu (CTU),Sichuan Airlines,320,C,3,International
2,3,06:50:00,SU634,Krasnoyarsk (KJA),Aeroflot,77W,E,5,International
3,4,06:55:00,DD3141,Chengdu (TFU),Nok Air,738,C,3,International
4,5,07:10:00,EY430,Abu Dhabi (AUH),Etihad Airways,B789 (A6-BLS),C,3,International
...,...,...,...,...,...,...,...,...,...
137,138,23:05:00,MU2813,Nanjing (NKG),China Eastern Airlines,A20N (B-306F),C,3,International
138,139,23:15:00,VZ316,Bangkok (BKK),Thai VietJet Air,320,C,3,Domestic
139,140,23:25:00,FD3023,Bangkok (DMK),Thai AirAsia,320,C,3,Domestic
140,141,23:30:00,FD525,Hong Kong (HKG),Thai AirAsia,320,C,3,International


In [3]:
#import Data of gate allocation rule
airport = pd.read_excel("Table_gate.xlsx",)
airport

,Apron,gate,ICAO code,max_size,flight_type
0,B,1,D,4,Domestic
1,B,2,D,4,Domestic
2,B,3,D,4,Domestic
3,B,4,D,4,Domestic
4,B,5,D,4,Domestic
5,A,6,E,5,Domestic
6,A,7,E,5,Domestic
7,A,8,E,5,Domestic
8,A,9,E,5,Domestic
9,A,10,E,5,Domestic


In [20]:
#นำ flight มาใส่ restriction ที่เกี่ยวกับ size และ ประเภท DO/IN แล้วทำ Dic เพื่อนำไปสร้าง genotype 
#แยกไม่ใส่ไว้ใน fitness เพื่อไม่ให้ frist Genaration เกิด infessible มากเกินไป

compatible_gates = {}
for idx, row in flights.iterrows():

    size_condition = airport.max_size >= row.plane_size #flight ใดๆลง gate ไหนได้บ้างตามขนาด wingspan
    
    flighttype_condition = airport.flight_type.str.contains(row.type) #แบ่ง Domestic กับ International

    combined_condition = size_condition & flighttype_condition

    gates_lst = airport[combined_condition].gate.values
    compatible_gates[row.Flights_no] = gates_lst
    
print("Compatible gates for each flights")
for k, v in compatible_gates.items():
    print(k, v)

Compatible gates for each flights
1 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
2 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
3 ['11' '12' '14' '31' '32' '33' '34']
4 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
5 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
6 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
7 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
8 ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '31' '32R' '32' '32L' '33R' '33'
 '33L' '34

In [12]:
import numpy as np
from random import random

gate = 5
flight = 10

class Individual():
 def __init__(self,generation=0):
    self.score_evaluation = 0
    self.generation = generation
    self.chromosome = np.random.randint(1, gate, flight)

 def fitness(self):
    score = 0
    for i in range(len(self.chromosome)):
      if self.chromosome[i] == 3:
         score += 1
    self.score_evaluation = score
      
 def crossover(self, other_individual):
    cutoff = round(random() * len(self.chromosome))
    print("Position of cutoff",cutoff)

    child1 = np.concatenate((self.chromosome[:cutoff], other_individual.chromosome[cutoff:]))
    child2 = np.concatenate((other_individual.chromosome[:cutoff], self.chromosome[cutoff:]))

    children = [Individual(self.generation + 1), Individual(self.generation + 1)]
    children[0].chromosome = child1
    children[1].chromosome = child2

    return children



In [13]:
individual1 = Individual()
individual1.fitness()

In [14]:
print('Chromosome: ', individual1.chromosome)
print('Score: ', individual1.score_evaluation)

Chromosome:  [3 4 4 4 3 1 4 3 4 4]
Score:  3


In [15]:
individual2 = Individual()
individual2.fitness()

In [16]:
print('Chromosome: ', individual2.chromosome)
print('Score: ', individual2.score_evaluation)

Chromosome:  [1 1 1 1 1 4 4 1 4 4]
Score:  0


In [17]:
children = individual1.crossover(individual2)

Position of cutoff 1


In [18]:
children[0].fitness()
print(children[0].score_evaluation)
print(children[0].chromosome)

1
[3 1 1 1 1 4 4 1 4 4]


In [19]:
children[1].fitness()
print(children[1].score_evaluation)
print(children[1].chromosome)

2
[1 4 4 4 3 1 4 3 4 4]
